# Building Molecular Hamiltonian
*Copyright (c) 2021 Institute for Quantum Computing, Baidu Inc. All Rights Reserved.*

## Overview

In this tutorial, we will demonstrate how to use Paddle Quantum's `qchem` module to build valid Hamiltonian for simulating chemical molecules on a quantum computer. We will go step by step how to build the second quantized Hamiltonian from a molecular structure and how to transform it to a set of Pauli matrices. 

Hamiltonian is a physical quantity related to the total energy of a physical system. In general, it can be represented as 

$$
\hat{H}=\hat{T}+\hat{V},\tag{1}
$$

where $\hat{T}$ is the kinetic energy and $\hat{V}$ is the potential energy. Hamiltonian is useful for various quantum algorithms, such as [variational quantum eigensolver](./VQE_EN.ipynb) and [Hamiltonian Simulation with Product Formula](./HamiltonianSimulation_EN.ipynb).

When trying to solve a chemistry problem with quantum mechanics, we also need to write down a Hamiltonian that describes the chemical system involved in the problem. Starting from this Hamiltonian, we can, in principle, calculate the ground state and excited states, and use the information to further explore all the physical properties of the quantum system. The dominant Hamiltonian of electronic problems has the form

$$
\hat{H}=\sum_{i=1}^N\left(-\frac{1}{2}\nabla_{x_i}^2\right)+\sum_{i=1}^N\sum_{j< i}\frac{1}{|x_i-x_j|}-\sum_{i=1}^N\sum_{I=1}^M\frac{Z_I}{|x_i-R_I|},\tag{2}
$$

when we use [atomic units](https://en.wikipedia.org/wiki/Hartree_atomic_units). Our electronic problem contains $N$ electrons and $M$ nucleus. We use $x_i$ to denote position of the $i$-th electron, and use $R_I$ to denote position of the $I$-th nuclei. 

This tutorial will have the following parts. Let's first talk about how to construct a molecule in `qchem`. After that, we will briefly describe how to calculate [Hartree Fock](https://en.wikipedia.org/wiki/Hartree%E2%80%93Fock_method) single particle orbitals by calling external quantum chemistry within Paddle Quantum. Next, we show how we can obtain the Hamiltonian in second quantization representation. Finally, we describe how to transform the Fermionic Hamiltonian to Pauli strings recognized by quantum computer.

## Defining the molecular structure
In this example, we show how to construct water molecule from its chemical formula and coordinates of atoms. 

![h2o.png](figures/buildingmolecule-fig-h2o.png)

Within Paddle Quantum, we specify the atom as a list whose first element is the atomic symbol and the second element is another list that contains its Cartesian coordinate. The molecule is thus a bigger list composed of atoms' list.

**Note: As to the environment setting, please refer to [README.md](https://github.com/PaddlePaddle/Quantum/blob/master/README.md).**

In [1]:
# Eliminate noisy python warnings
import warnings

warnings.filterwarnings("ignore")

In [2]:
# in Angstrom
h2o_structure_direct = [["H", [-0.02111417,0.8350417,1.47688078]],  # H stands for hydrogen element in water
                        ["O", [0.0, 0.0, 0.0]],                     # O stands for oxygen element in water
                        ["H", [-0.00201087,0.45191737,-0.27300254]]]

Instead of specifying molecular structure directly, we can also pass the \*.xyz file to the `geometry` function to get the same structure.

In [4]:
from paddle_quantum.qchem import geometry

h2o_structure_xyz = geometry(file="h2o.xyz")
assert h2o_structure_xyz == h2o_structure_direct

## Calculate Hartree Fock orbitals
Hartree Fock method uses the [Slater determinant](https://en.wikipedia.org/wiki/Slater_determinant) to represent the $N$-electron wavefunction. It could provide us with a set of single particle orbitals which are often taken as input to more advanced quantum chemistry methods. 

Paddle Quantum uses psi4 [1] as its quantum chemistry engine. We could use the `get_molecular_data` function provided in `qchem` module to manage the quantum chemistry calculation and get the necessary information about the molecule. `get_molecular_data` function takes molecular structure, total molecular charge, and spin multiplicity as its major inputs, it will return an OpenFermion [2] `MolecularData` object. 

Let's continue with our water molecule example. To run the Hartree Fock calculation, we need to set the `method` keyword argument to *scf* (Self Consistent Field). We can also improve the quality of Hartree Fock calculation by specifying the type of [basis set](https://en.wikipedia.org/wiki/Basis_set_(chemistry)) in the `basis` argument. 

In [ ]:
from paddle_quantum.qchem import get_molecular_data

h2o_moledata = get_molecular_data(
    h2o_structure_direct,
    charge=0,                # Water molecule is charge neutral
    multiplicity=1,          # In the ground state, the lowest 5 molecular orbitals of water molecular will be occupied by a pair of electrons with opposite spin
    basis="sto-3g",
    method="scf",
    if_save=True,            # Whether to save information contained in MolecularData object to a hdf5 file
    if_print=True,           # Wheter to print the ground state energy of water molecule
    name="",                 # Specifies the name of the hdf5 file
    file_path="."            # Specifies where to store the hdf5 file          
)

from openfermion.chem import MolecularData

assert isinstance(h2o_moledata, MolecularData)

## Molecular Hamiltonian in second quantization form
When we study many electron quantum systems, it's often convenient to write Hamiltonian at the beginning of this tutorial in [second quantization](https://en.wikipedia.org/wiki/Second_quantization) representation 

$$
\hat{H}=\sum_{p,q}h_{pq}\hat{c}^{\dagger}_p\hat{c}_q+\frac{1}{2}\sum_{p,q,r,s}v_{pqrs}\hat{c}^{\dagger}_p\hat{c}^{\dagger}_q\hat{c}_r\hat{c}_s,\tag{3}$$

where $p$, $q$, $r$ and $s$ are Hartree Fock orbitals computed in the previous section. $\hat{c}^{\dagger}_p$ and $\hat{c}_q$ are creation and annihilation operations, respectively. The two coefficients $h_{pq}$ and $v_{pqrs}$ are called molecular integrals, and can be obtained from `MolecularData` object in the following way.

In [6]:
import numpy as np 
np.set_printoptions(precision=4, linewidth=150)

hpq, vpqrs = h2o_moledata.get_integrals()
assert np.shape(hpq)==(7, 7)             # When use sto3g basis, the total number of molecular orbitals used in water calculation is 7
assert np.shape(vpqrs)==(7, 7, 7, 7)

print(hpq)
# print(vpqrs)

[[-3.2911e+01  5.5623e-01  2.8755e-01  1.4640e-15 -7.4568e-02 -9.4552e-02  2.8670e-01]
 [ 5.5623e-01 -8.0729e+00 -4.0904e-02 -1.6823e-15  1.7890e-01  3.5048e-01 -1.3460e+00]
 [ 2.8755e-01 -4.0904e-02 -7.3355e+00 -4.8424e-15  4.1911e-01  5.2109e-01  7.0928e-01]
 [ 1.4640e-15 -1.6823e-15 -4.8424e-15 -7.5108e+00 -1.4127e-14 -2.6576e-14 -1.5008e-15]
 [-7.4568e-02  1.7890e-01  4.1911e-01 -1.4127e-14 -5.7849e+00  2.0887e+00  1.2427e-01]
 [-9.4552e-02  3.5048e-01  5.2109e-01 -2.6576e-14  2.0887e+00 -5.0803e+00  1.3967e-02]
 [ 2.8670e-01 -1.3460e+00  7.0928e-01 -1.5008e-15  1.2427e-01  1.3967e-02 -5.0218e+00]]


Most of the time, we don't need to extract those integrals and assemble the Hamiltonian manually, *qchem* module has already helped us take care of this by providing the `fermionic_hamiltonian` function.

In [7]:
from paddle_quantum.qchem import fermionic_hamiltonian

H_of_water = fermionic_hamiltonian(
    h2o_moledata,
    multiplicity=1,
    active_electrons=4,
    active_orbitals=4
)

from openfermion.ops import FermionOperator

assert isinstance(H_of_water, FermionOperator)

By specifying `active_electrons` and `active_orbitals` keyword arguments, we can reduce the number of freedom of our Hamiltonian and thus reduce the number of terms in the spin Hamiltonian described in the next section. We can also use `active_space` function in *qchem* to return a list of *core* orbitals and *active* orbitals. 

In [8]:
from paddle_quantum.qchem import active_space

core_orbits_list, act_orbits_list = active_space(
    10,                        # number of electrons in water molecule
    7,                         # number of molecular orbitals in water molecule
    active_electrons=4,
    active_orbitals=4
)

print("List of core orbitals: {:}".format(core_orbits_list))
print("List of active orbitals: {:}".format(act_orbits_list))

List of core orbitals: [0, 1, 2]
List of active orbitals: [3, 4, 5, 6]


## From Fermionic Hamiltonian to spin Hamiltonian
In quantum computing, we only have qubit operators composed of Pauli matrices

$$
\boldsymbol{\sigma}_x=\begin{pmatrix}
0 & 1\\
1 & 0
\end{pmatrix},\quad \boldsymbol{\sigma}_y=\begin{pmatrix}
0 & -i\\
i & 0
\end{pmatrix},\quad \boldsymbol{\sigma}_z=\begin{pmatrix}
1 & 0\\
0 & -1
\end{pmatrix}.\tag{4}
$$

Therefore, we need to transform our Hamiltonian in the previous section to qubit operators, [Jordan-Wigner transform](https://en.wikipedia.org/wiki/Jordan%E2%80%93Wigner_transformation) is one of the well-known methods to realize the transformation.
> Alternatively, we also provide Bravyi-Kitaev transformation, by changing the argument, mapping_method, to 'bravyi_kitaev'.

In *paddle quantum*, Hamiltonian is encoded in *pauli_str*. To avoid tedious manipulation of *string* object, we have provided `spin_hamiltonian` function which can generate the needed *pauli_str* from molecular structure on the fly.

In [9]:
from paddle_quantum.qchem import spin_hamiltonian

pauli_H_of_water_ = spin_hamiltonian(
    h2o_moledata,
    multiplicity=1,
    active_electrons=4,
    active_orbitals=4,
    mapping_method='jordan_wigner'
)

print('There are ', pauli_H_of_water_.n_terms, 'terms in H2O Hamiltonian in total.')
print('The first 10 terms are \n', pauli_H_of_water_[:10])

There are  193 terms in H2O Hamiltonian in total.
The first 10 terms are 
 -72.10615980544185 I
-0.007310917992546774 X0, X1, Y2, Y3
0.0052460870730834325 X0, X1, Y2, Z3, Z4, Y5
0.0016283548447087654 X0, X1, Y2, Z3, Z4, Z5, Z6, Y7
0.0052460870730834325 X0, X1, X3, X4
0.0016283548447087654 X0, X1, X3, Z4, Z5, X6
-0.005994544380559027 X0, X1, Y4, Y5
0.001387644178102622 X0, X1, Y4, Z5, Z6, Y7
0.001387644178102622 X0, X1, X5, X6
-0.009538223793221182 X0, X1, Y6, Y7


Great! Now you know how to build a proper Hamiltonian from a given molecular structure, let's move further and see how to use [variational quantum eigensolver](./VQE_EN.ipynb) (VQE) to determine the ground state of hydrogen molecule.


---
## References

[1] [Psi4: an open-source ab initio electronic structure program](https://wires.onlinelibrary.wiley.com/doi/abs/10.1002/wcms.93)

[2] [OpenFermion: the electronic structure package for quantum computers
](https://iopscience.iop.org/article/10.1088/2058-9565/ab8ebc)